In [3]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 115.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 116.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=efe5ad562c1b57c7ab8f27aabe051c7a2e515e5701731a04028f236a0d89c876
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [4]:

!pip install nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [6]:
!pip install scipy
import scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
traits = [['responsible', 'encourage', 'accountable', 'help', 'ownership', 'support', 'trained', 'qualified', 'understand', 'non-cited', 'involvement', 'self-identifying'],
['complacent', 'discrepancy', 'anomaly', 'assumption', 'question', 'uncertain', 'unknown', 'risk', 'investigate', 'unexpected', 'unclear', 'review', 'degrading', 'aging'],
['communication', 'information', 'safety', 'share', 'respond', 'listen', 'concerns', 'expectations', 'clear'],
['leadership', 'management', 'leaders', 'owner', 'guidance', 'decisions', 'policies', 'resources', 'staffing', 'expectations', 'oversight', 'reinforce', 'decision-making', 'mentoring', 'coaching', 'encourage', 'incentive', 'reward', 'priorities', 'plan', 'delegate', 'define', 'manage', 'support', 'resolve', 'motivate', 'address', 'resources', 'translate', 'funding', 'implementation'],
['decision-making', 'thorough', 'conservative', 'systematic', 'consistent', 'process', 'leaders', 'choices', 'consequences', 'reinforce', 'accountable', 'authority', 'responsibility'],
['trust', 'respect', 'encourage', 'opinions', 'bullying', 'fair', 'productive', 'concerns', 'voice', 'discussions', 'value', 'insight', 'perspectives', 'collaboration', 'conflict', 'listening'],
['learn', 'training', 'self-assessments', 'improve', 'performance', 'monitor', 'experience', 'assessment', 'corrective action', 'benchmarking', 'knowledge', 'competence', 'skills', 'develop', 'understand'],
['identify', 'address', 'correct', 'corrective action', 'issues', 'deviation', 'degraded conditions', 'evaluation', 'problems', 'root cause', 'investigation', 'recommendation', 'cause analysis', 'resolution', 'mitigate', 'trends'],
['concerns', 'confidence', 'address', 'feedback', 'listening', 'report'],
['plan', 'control', 'activities', 'production', 'schedule', 'manage', 'execute', 'risk', 'coordinate', 'margins', 'operate', 'maintain', 'maintenance', 'document', 'complete', 'procedures', 'packages', 'accurate', 'current', 'backlog', 'follow', 'review', 'instructions', 'operations', 'design', 'requirements', 'operations']]

In [9]:
for i in range(len(traits)):
  traits[i] = ' '.join(a for a in traits[i])

In [10]:
import json
# 24th report
f = open('Extracted_Negative_Sentences.json')

data = json.load(f)
  
sentences = []

for i in data:
    sentence = i['sentence']
    if sentence not in sentences:
        sentences.append(sentence)


In [12]:
import re
import spacy

nlp = spacy.load('en_core_web_sm')
stop_words = nlp.Defaults.stop_words

for i in range(len(sentences)):
        
  text = sentences[i]

  text = re.sub('{January|February|March|April|May|June|July|August|September|October|November|December}', '', text)

  lemma_list = [token.lemma_ for token in nlp(text)]

  clean_list = [re.sub('-PRON-', '', word) for word in lemma_list if word not in stop_words]

  clean_text = ' '.join(clean_list).lower()

  clean_text=re.sub(r'\d+', '', clean_text)
  clean_text=re.sub(r'[\W_]+', ' ', clean_text).rstrip()

  terms_list = [word for word in clean_text.split() if len(word) > 2]
  clean_text = ' '.join(terms_list).lower()
  sentences[i] = clean_text


In [20]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Sentences are encoded by calling model.encode()
embeddings = model.encode(traits)


In [28]:
from prettytable import PrettyTable

table = PrettyTable(["Sentence","Trait","Distance"])

traits = ['Personal Accountability','Questioning Attitude', 'Safety Communication', 'Leadership Values', 'Decision Making', 'Respectful Environment', 
          'Continuous Learning', 'Problem Identification', 'Raising Concerns', 'Work Processes']

for a in sentences:
  sent = [a for _ in range(10)]
  emb = model.encode(sent)
  distances = scipy.spatial.distance.cdist(embeddings, emb, "cosine")
  maxer,ind = float('inf'),0
  # print(distances)
  for i in range(10):
    if maxer>distances[i][0]:
      ind,maxer = i,distances[i][0]
  table.add_row([a, traits[ind], maxer])

print(table)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------